In [1]:
import pymarc as pymarc
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

class NeuralNet:
    
    def __init__(self):
        pass        

class Node:
    
    def __init__(self, values, weights, bias, aggregation, activation):
        self._values = values
        self._weights = weights
        self._bias = bias
        self._activation = activation
    
    def _get_values(self):
        return self._values
    
    def _set_values(self, values):
        self._values = values
    
    values = property(_get_values, _set_values)
    
    def _aggregate(self):
        return aggregate(self._values, self._weights, self._bias)
    
    def activate(self):
        return activate(Node._aggregate(self))
    
            
        
        
def aggregate(values, weights, bias):
    return np.dot(values, weights) + bias * 1

def activate(aggregation):
    return 1 / (1 + np.exp(-aggregation))

def compute_total_error(target, output):
    return np.sum(1/2 * (target - output)**2)

def compute_e_partial_o1(target, output):
    return 2 * 1/2 * (target - output) * -1

def compute_o1_partial_net(output):
    return output * (1 - output)


In [2]:
# Input values
input = np.array([0.05, 0.1])

# Input weights + bias
input_weights_1 = np.array([0.15, 0.2])
input_weights_2 = np.array([0.25, 0.3])
bias_1 = np.array([0.35])

# Hidden layer weights + bias
hidden_weights_1 = np.array([.4, .45])
hidden_weights_2 = np.array([.5, .55])
bias_2 = np.array([.6])


h1 = Node(input, input_weights_1, bias_1, aggregate, activate)
h2 = Node(input, input_weights_2, bias_1, aggregate, activate)

h_out = np.concatenate([h1.activate(), h2.activate()])

o1 = Node(h_out, hidden_weights_1, bias_2, aggregate, activate)
o2 = Node(h_out, hidden_weights_2, bias_2, aggregate, activate)

o_out = np.concatenate([o1.activate(), o2.activate()])

e_o1 = compute_total_error(0.01, o1.activate())
e_o2 = compute_total_error(0.99, o2.activate())

e_total = e_o1 + e_o2

e_partial_o1 = compute_e_partial_o1(0.01, o1.activate())
o1_partial_net = compute_o1_partial_net(o1.activate())
o1_partial_w5 = 1 * h1.activate()
w5 = 0.4 - .5 * e_partial_o1 * o1_partial_net * o1_partial_w5

#0.4 - 0.5 * 0.082167041
w5




array([ 0.35891648])